In [1]:
import numpy as np
import pandas as pd
import pyodbc as pyo
import sqlalchemy as sa
import numpy as np
import math
import datetime
import datetime as dt
connection_uri = (
    "mssql+pyodbc://PHLHCMHOPPBI02\PBIRS/PowerBI?driver=ODBC+Driver+17+for+SQL+Server"
)
conn = sa.create_engine(connection_uri)

<h4>1. Lấy bảng<h4>

<h4> 1.1 Lấy bảng Premium

In [2]:
cutoff = '20230131'

In [3]:
premium_query = f"""\
WITH CUTOFF_TABLE AS (
SELECT DISTINCT CUTOFF
  FROM [DPO].[DP_TAGENTPREMIUM_TEST_CUTOFF])

, CUTOFF_1 AS (SELECT *, LEFT(CUTOFF, 6) AS MONTH_CUTOFF, 
ROW_NUMBER() OVER (PARTITION BY LEFT(CUTOFF, 6) ORDER BY CUTOFF DESC) AS PRI_NO
FROM CUTOFF_TABLE )

, CUTOFF_FINAL AS (SELECT CUTOFF, 
LEAD(CUTOFF, 1) OVER (ORDER BY MONTH_CUTOFF DESC ) AS LAST_CUTOFF
FROM CUTOFF_1 
WHERE PRI_NO = 1)

, PREMIUM_CURRENT AS (
SELECT * FROM [DPO].[DP_TAGENTPREMIUM_TEST_CUTOFF])

, PREMIUM_LAST AS (
SELECT DISTINCT [Policy No], [Policy Status], CUTOFF, CONCAT([Policy No], CUTOFF) as Link_Key_Prev FROM [DPO].[DP_TAGENTPREMIUM_TEST_CUTOFF])

, CURRENT_PRE AS (SELECT a.*, CONCAT([Policy No], LAST_CUTOFF) AS Link_Key FROM PREMIUM_CURRENT a
LEFT JOIN CUTOFF_FINAL b
on a.CUTOFF = b.CUTOFF)

select a.*, b.[Policy Status] as Last_Policy_STT from CURRENT_PRE a
LEFT JOIN PREMIUM_LAST b
on a.Link_Key = b.Link_Key_Prev 
where a.CUTOFF = '{cutoff}'"""
premium = pd.read_sql(premium_query, conn)

In [4]:
premium_table = premium

<h4>1.2 Lấy bảng DAILYSALES



In [5]:
daily_duong_query = f"""\
SELECT [Agent Code], [Policy_Number],[Issuing_Agent],[Contract Type],[Component_Code],[Proposal_Receive_Date]
      ,[Policy_Issue_Date] ,[Sum_Assured],[Before_Discount_Premium],[Discount_Premium],[After_Discount_Premium],[Policy_Status]
      ,[Bill_Frequency],[Modal_Factor],[Lapsed_date],[AFYP],[RISK_COMMENCE_DATE],[CUTOFF],[ACK_DATE] 
FROM [DPO].[DP_TDAILYSALES_DA_CUTOFF]
where CUTOFF = '{cutoff}' """
daily_duong = pd.read_sql(daily_duong_query, conn)

In [6]:
daily_am_query = f"""\
SELECT [Agent Code], [Policy_Number],[Issuing_Agent],[Contract Type],[Component_Code],[Proposal_Receive_Date]
      ,[Policy_Issue_Date] ,[Sum_Assured],[Before_Discount_Premium],[Discount_Premium],[After_Discount_Premium],[Policy_Status]
      ,[Bill_Frequency],[Modal_Factor],[Lapsed_date],[AFYP],[RISK_COMMENCE_DATE],[CUTOFF],[ACK_DATE] 
FROM [DPO].[DP_DA_Daily_DC_PO_WD_NT_CUTOFF]
where CUTOFF = '{cutoff}' """
daily_am = pd.read_sql(daily_am_query, conn)

In [7]:
salesdaily = pd.concat([daily_duong, daily_am])

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_19252\796227512.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  salesdaily = pd.concat([daily_duong, daily_am])


<h4> 1.3 Lấy bảng PY2 

In [8]:
py2_query = f"select * from [DPO].[DP_K2_CUTOFF] where CUTOFF = '{cutoff}'"
py2 = pd.read_sql(py2_query, conn)

<h4> 1.4 Lấy bảng Agent_Info

In [9]:
agent_query = f"select * from [DPO].[Main_AGENT_INFO_DA_CUTOFF] where CUTOFF = '{cutoff}'"
agent_info = pd.read_sql(agent_query, conn)

<h4> 2. Lấy ra PERIOD<h4>

In [10]:
premium_table = premium_table.loc[:, premium_table.columns != 'Link_Key']

In [11]:
#Thêm cột ngày xét
Monthly_Premium = premium_table
Monthly_Premium['Date_On_Set'] = "" #Tạo ra cột trống
Monthly_Premium['Date_On_Set']= Monthly_Premium['Date_On_Set'].str.cat(Monthly_Premium['CUTOFF'].str[:4] ,sep ="")
Monthly_Premium['Date_On_Set']= Monthly_Premium['Date_On_Set'].str.cat(Monthly_Premium['CUTOFF'].str[4:6] ,sep ="-")
Monthly_Premium['Date_On_Set']= Monthly_Premium['Date_On_Set'].str.cat(Monthly_Premium['CUTOFF'].str[6:8] ,sep ="-")

In [12]:
#Phải biến thành kiểu datetime trước
Monthly_Premium['Date_On_Set'] = pd.to_datetime(Monthly_Premium['Date_On_Set'])
#Thêm cột ngày đầu tháng:
#-> Đưa vào ngày 1 của tháng
#LƯU Ý CÁC NGÀY OFF ĐẶC BIỆT CỦA THÁNG 4.2023 VÀ 01/2024
if not('20240229' in Monthly_Premium['CUTOFF'].values): 
    Monthly_Premium['Start_Month'] = (Monthly_Premium['Date_On_Set'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))
elif ('20230531' in Monthly_Premium['CUTOFF'].values):
    Monthly_Premium.loc[:,'Start_Month'] = '2023-04-29'
else:
    Monthly_Premium.loc[:,'Start_Month'] = '2024-01-27'

In [13]:
#Lấy ra ngày Period:
#LƯU Ý CÁCH CỘNG 1 NGÀY VÀO CỘT APPLIED PREMIUM DATE
Period_Com = Monthly_Premium
Period_Com ['Date_Period_1'] = np.where( (Period_Com['Policy Status'] == 'FL'), Period_Com['Issued Date'],
np.where( (Period_Com['POLICY ACKNOWLED'].isnull()), Period_Com['Issued Date'],
np.where((((Period_Com['Applied Premium Date'] + datetime.timedelta(days=1)) - Period_Com['POLICY ACKNOWLED']) / np.timedelta64(1, 'D')) < 22,
Period_Com['Issued Date'], Period_Com['Applied Premium Date'])))

In [14]:
Period_Prem = Period_Com
calculate_condition = ((Period_Prem['Applied Premium Date'] >= Period_Prem['Start_Month']) & (Period_Prem['Applied Premium Date'] <= Period_Prem['Date_On_Set']))
PREM_month = Period_Prem[((Period_Prem['Applied Premium Date'] >= Period_Prem['Start_Month']) & (Period_Prem['Applied Premium Date'] <= Period_Prem['Date_On_Set']))]

In [15]:
PREM_month['Date_Period'] = np.where((PREM_month['Date_Period_1'].isnull()), PREM_month['Applied Premium Date'], PREM_month['Date_Period_1'])

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_19252\3346513835.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PREM_month['Date_Period'] = np.where((PREM_month['Date_Period_1'].isnull()), PREM_month['Applied Premium Date'], PREM_month['Date_Period_1'])


In [16]:
PREM_month['Period_Month'] = pd.DatetimeIndex(PREM_month['Date_Period']).month
PREM_month['Period_Year'] = pd.DatetimeIndex(PREM_month['Date_Period']).year
PREM_month['Period'] = np.where(PREM_month['Period_Month'] >9, PREM_month['Period_Year'].astype('str') + PREM_month['Period_Month'].astype('str'),
PREM_month['Period_Year'].astype('str') + "0" +PREM_month['Period_Month'].astype('str')                              
                               )

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_19252\4075263784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PREM_month['Period_Month'] = pd.DatetimeIndex(PREM_month['Date_Period']).month
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_19252\4075263784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PREM_month['Period_Year'] = pd.DatetimeIndex(PREM_month['Date_Period']).year
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_19252\4075263784.py:3: SettingWithCopyWarning: 
A value is trying to be set o

<h4>3. Tính CaseCount <h4>

In [17]:
CC_prem = PREM_month
CC_prem['Status Transition'] = CC_prem['Last_Policy_STT'] + "to" + CC_prem['Policy Status'] 
#Loại bỏ khoản trắng:
CC_prem['PREM TRAN NAME'] = CC_prem['PREM TRAN NAME'].str.strip()

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_19252\1393532758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CC_prem['Status Transition'] = CC_prem['Last_Policy_STT'] + "to" + CC_prem['Policy Status']
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_19252\1393532758.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CC_prem['PREM TRAN NAME'] = CC_prem['PREM TRAN NAME'].str.strip()


In [18]:
CC_prem['Count_Stage_1'] = np.where((CC_prem['PREM TRAN NAME'] == 'Contract Issue') & (CC_prem['Last_Policy_STT'].isnull()) , 1,
                        np.where((CC_prem['PREM TRAN NAME'] == 'Freelook Cancellation') & (CC_prem['Last_Policy_STT'].isnull()),-1, 0))

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_19252\3210236533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CC_prem['Count_Stage_1'] = np.where((CC_prem['PREM TRAN NAME'] == 'Contract Issue') & (CC_prem['Last_Policy_STT'].isnull()) , 1,


In [19]:
CC_prem['Count_Stage_2'] = np.where((CC_prem['Status Transition'].str.len() >5) &\
                        CC_prem['Status Transition'].str[4:].isin(['FL', 'CF', 'DC', 'DH', 'NT', 'PO', 'WD']), -1,
                        np.where((CC_prem['Status Transition'].str.len() < 5) &\
                        CC_prem['Status Transition'].str[4:].isin(['IF']), 1, 
                        np.where((CC_prem['Policy Status'].isin(['CF', 'DC', 'NT', 'PO', 'WD'])), -1, 0    
                                )))

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_19252\2212519876.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CC_prem['Count_Stage_2'] = np.where((CC_prem['Status Transition'].str.len() >5) &\


In [20]:
CC_prem['Count_Policy'] = np.where((CC_prem['Count_Stage_2']) == 0, CC_prem['Count_Stage_1'], CC_prem['Count_Stage_2'] )

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_19252\3545740601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CC_prem['Count_Policy'] = np.where((CC_prem['Count_Stage_2']) == 0, CC_prem['Count_Stage_1'], CC_prem['Count_Stage_2'] )


<h4>4. Sum để lấy CaseCount <h4>

In [21]:
#Nếu muốn check trạng thái CC của HD, check chỗ này
CaseCount = CC_prem
CaseCount = CaseCount[['Issuing Agent', 'Policy No', 'CUTOFF', 'Count_Policy']].drop_duplicates()
Sum_CC = CaseCount.groupby(['Issuing Agent', 'Policy No', 'CUTOFF'])['Count_Policy'].sum()
Sum_CC = Sum_CC.reset_index()

In [22]:
Sum_CC.loc[(Sum_CC['Count_Policy'] <0),'CC(-)'] = Sum_CC['Count_Policy']
Sum_CC.loc[(Sum_CC['Count_Policy'] >0),'CC(+)'] = Sum_CC['Count_Policy']

<h5>4.1 Sum lại CaseCount theo Đại lý phát hành HD <h4>

In [23]:
Agent_CC = Sum_CC.groupby(['Issuing Agent', 'CUTOFF'])[['CC(-)', 'CC(+)']].sum()
Agent_CC = Agent_CC.reset_index()

In [24]:
Agent_CC

,Issuing Agent,CUTOFF,CC(-),CC(+)
0,60000073,20230131,0.0,0.0
1,60000156,20230131,0.0,0.0
2,60000204,20230131,0.0,0.0
3,60000482,20230131,0.0,1.0
4,60000717,20230131,0.0,1.0
...,...,...,...,...
3016,69999988,20230131,0.0,0.0
3017,69999989,20230131,0.0,0.0
3018,69999993,20230131,0.0,0.0
3019,69999996,20230131,0.0,0.0


<h4>5. Tính FYP <h4>

In [25]:
FYP_month = PREM_month
FYP_month = pd.merge(FYP_month, Sum_CC, left_on = 'Policy No', right_on = 'Policy No', how = 'left' )
FYP_month = FYP_month[['Policy No', 'Product Code', 'Premium transaction', 'PREM TRAN NAME',
       'Premium Collected', 'Collected Date', 'Applied Premium Date',
       'FYP Before Discount', 'FYP Discount', 'FYP', 'RYP', 'Topup Premium',
       'Premium Term', 'Premium Year', 'Policy Status', 'Policy Year',
       'Policy Term', 'Frequency of Payment', 'Issued Date', 'Effected Date',
       'Terminated Date', 'Lapse Date', 'Due date', 'Next Due Date',
       'Transfer Date', 'POLICY ACKNOWLED', 'Sum Assure', 'Area Code',
       'Servicing Agent', 'Freelook', 'Proposal Receive Date',
       'RISK_COMMENCE_DATE', 'Age_Customer', 'Issuing Agent_x', 'AFYP',
       'CUTOFF_x', 'RETURN_DATE', 'Count_Policy_y']].rename(columns={'Issuing Agent_x': 'Issuing Agent', 'Count_Policy_y': 'Count_Policy','CUTOFF_x':'CUTOFF'})

<h5> 5.1 Sum lại theo Servicing Agent lấy FYP <h5>

In [26]:
Sum_FYP = FYP_month
Sum_FYP['FYP_Total'] = Sum_FYP['FYP'] + Sum_FYP['Topup Premium']*0.1
Sum_FYP = Sum_FYP.groupby(['Servicing Agent','Policy No', 'CUTOFF', 'Count_Policy'])['FYP_Total'].sum()
Sum_FYP = Sum_FYP.reset_index()
#Rules: Các HD Mark -1=> HD giảm phí, tính FYP(-), các HD đang còn IF, trạng thái 0, 1 => FYP(+)
Sum_FYP.loc[(Sum_FYP['Count_Policy'] <0), 'FYP(-)'] = Sum_FYP['FYP_Total']
Sum_FYP.loc[(Sum_FYP['Count_Policy'] >=0), 'FYP(+)'] = Sum_FYP['FYP_Total']
Sum_FYP = Sum_FYP.groupby(['Servicing Agent'])[['FYP(-)', 'FYP(+)']].sum()
Sum_FYP

,FYP(-),FYP(+)
Servicing Agent,,
60000194,0.0,0.0
60000449,0.0,0.0
60000482,0.0,16026000.0
60000564,0.0,0.0
60000717,0.0,16141000.0
...,...,...
60076140,0.0,10003000.0
60076141,0.0,30256000.0
60076148,0.0,32048000.0


<h5>5.2 Tính RYP <h5>

In [27]:
Sum_RYP = FYP_month
Sum_RYP = Sum_RYP.groupby(['Servicing Agent'])['RYP'].sum()
Sum_RYP = Sum_RYP.reset_index()
Sum_RYP

,Servicing Agent,RYP
0,60000194,9626000.0
1,60000449,19390000.0
2,60000482,0.0
3,60000564,5965000.0
4,60000717,0.0
...,...,...
2650,60076140,0.0
2651,60076141,0.0
2652,60076148,0.0
2653,69999997,24683000.0


<h4> 6. Tính AFYP <h4>

<h5> 6.1 Sum lại AFYP <h4>

In [28]:
AFYP = salesdaily
Sum_AFYP = AFYP
Sum_AFYP = Sum_AFYP.groupby(['Policy_Number'])['AFYP'].sum()
Sum_AFYP = Sum_AFYP.reset_index()

In [29]:
Sum_AFYP

,Policy_Number,AFYP
0,80000003,196736000.0
1,80000006,42400000.0
2,80000007,42400000.0
3,80000008,57664000.0
4,80000012,11332000.0
...,...,...
128682,80133067,20235000.0
128683,80133068,12238000.0
128684,80133069,0.0
128685,80133070,19854000.0


<h5> 6.2 Sum lại Topup để cộng vào AFYP <h4>

In [30]:
Topup_month = FYP_month #đoạn này đã xong phần CC, FYP (-), FYP (+)
Topup_month = Topup_month.groupby(['Policy No', 'Count_Policy'])['Topup Premium'].sum()
Topup_month = Topup_month.reset_index()
Topup_month

,Policy No,Count_Policy,Topup Premium
0,80000077,0,0.0
1,80000089,0,0.0
2,80000222,0,0.0
3,80000292,0,0.0
4,80000301,0,0.0
...,...,...,...
4400,80133066,1,0.0
4401,80133067,1,0.0
4402,80133068,1,0.0
4403,80133070,1,0.0


<h5>6.3 Tính tổng AFYP bao gồm Top_Up <h4>

In [31]:
AFYP_Topup = pd.merge(Sum_AFYP, Topup_month, left_on = 'Policy_Number', right_on ='Policy No', how = 'left')
AFYP_Topup = AFYP_Topup[['Policy_Number', 'AFYP', 'Topup Premium', 'Count_Policy']].fillna(0)
AFYP_Topup['AFYP_Topup'] = np.where(AFYP_Topup['Count_Policy'] >=0 , AFYP_Topup['AFYP'] + AFYP_Topup['Topup Premium'] *0.1,
                           AFYP_Topup['AFYP'] + AFYP_Topup['Topup Premium'] *-0.1)     
AFYP_Topup = AFYP_Topup[['Policy_Number', 'Count_Policy', 'AFYP_Topup']]
AFYP_Topup['AFYP_Topup'] = AFYP_Topup['AFYP_Topup'].abs()
AFYP_Topup

,Policy_Number,Count_Policy,AFYP_Topup
0,80000003,0.0,196736000.0
1,80000006,0.0,42400000.0
2,80000007,0.0,42400000.0
3,80000008,0.0,57664000.0
4,80000012,0.0,11332000.0
...,...,...,...
128682,80133067,1.0,20235000.0
128683,80133068,1.0,12238000.0
128684,80133069,0.0,0.0
128685,80133070,1.0,19854000.0


<h5> 6.4 Lấy thông tin Servicing Agent với HD <h4>

In [32]:
Final_AFYP = FYP_month[['Servicing Agent', 'Policy No']].drop_duplicates()
Final_AFYP = pd.merge(AFYP_Topup, Final_AFYP, left_on = 'Policy_Number', right_on = 'Policy No', how = 'left')
Final_AFYP = Final_AFYP[['Servicing Agent', 'Policy_Number', 'Count_Policy', 'AFYP_Topup']]

<h5>6.5 Sum kết quả  AFYP lại (Nếu check AFYP thì check ở đây)

In [33]:
Complete_AFYP = Final_AFYP
Complete_AFYP.loc[Complete_AFYP['Count_Policy'] >0, 'AFYP(+)'] = Complete_AFYP['AFYP_Topup']
Complete_AFYP.loc[Complete_AFYP['Count_Policy'] <0, 'AFYP(-)'] = Complete_AFYP['AFYP_Topup'] * -1
Complete_AFYP = Complete_AFYP.groupby(['Servicing Agent'])[['AFYP(+)', 'AFYP(-)']].sum()
Complete_AFYP = Complete_AFYP.reset_index()

<h4>7. Tính PY2 <h4>

In [34]:
py2_table = py2

In [35]:
py2_month = py2_table.groupby(['AGENT NO'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum()
py2_month['PY2'] = np.where((py2_month['Y2 ACTUAL PREM'] / py2_month['Y2 EXPECTED PREM']) > 1, 1, 
py2_month['Y2 ACTUAL PREM'] / py2_month['Y2 EXPECTED PREM'])
py2_month['PY2'] = round(py2_month['PY2'] , 2)
py2_month = py2_month.reset_index()

In [36]:
py2_month

,AGENT NO,Y2 ACTUAL PREM,Y2 EXPECTED PREM,PY2
0,60000450,111587000.0,111587000.0,1.00
1,60000580,4236000.0,8437000.0,0.50
2,60000597,0.0,22714000.0,0.00
3,60001375,0.0,17860000.0,0.00
4,60001457,28758000.0,34098000.0,0.84
...,...,...,...,...
6120,69999896,41397000.0,69193000.0,0.60
6121,69999897,0.0,16314000.0,0.00
6122,69999899,8015000.0,23108000.0,0.35
6123,69999997,221168000.0,201938000.0,1.00


<h4> 8. Tính Base và Rider <h4>

<h5> 8.1 Check phân loại Base/ Rider tại đây!

In [43]:
base_rider = PREM_month
base_rider = base_rider[['Issuing Agent','Policy No' , 'Product Code']].rename(columns= {'Policy No': 'PolicyNum', 
                                                                            'Issuing Agent': 'Agent_Issue'})
#Join lấy STT hợp đồng
base_rider = pd.merge(base_rider, Sum_CC, left_on = 'PolicyNum', right_on = 'Policy No', how = 'left')
base_rider = base_rider[['Issuing Agent', 'PolicyNum', 'Product Code', 'Count_Policy']]
base_rider = base_rider[base_rider['Count_Policy'] >0]
base_rider['Base/Rider'] = np.where(base_rider['Product Code'].isin(['EC01', 'EC02', 'EE01', 'EE02', 'EE03', 'EE04', 'EN03', 'EN04', 'EN05', 'EN06', 'EN07',\
'ES01', 'TL01', 'UL01', 'UL02', 'UL03', 'UL04', 'UL05', 'UL06', 'UL81', 'UL82', 'UL83']), 'Base', 'Rider')
base_rider

,Issuing Agent,PolicyNum,Product Code,Count_Policy,Base/Rider
2660,60058033,80133017,RHB1,1,Rider
2661,60058033,80133017,URC6,1,Rider
2662,60058033,80133017,URA5,1,Rider
2663,60058033,80133017,UL05,1,Base
2664,60058033,80133017,RHS1,1,Rider
...,...,...,...,...,...
18744,60075549,80131615,RHB1,1,Rider
18745,60075549,80131615,URC5,1,Rider
18752,60075604,80132232,RHS1,1,Rider
18753,60075604,80132232,URC5,1,Rider


In [44]:
#Loại bỏ dòng trùng để lấy kết quả cuối cùng
agent_pol = base_rider[['Issuing Agent', 'PolicyNum']].drop_duplicates()
base_rider = base_rider[['PolicyNum', 'Product Code', 'Count_Policy', 'Base/Rider']].drop_duplicates()
base_rider = pd.merge(agent_pol, base_rider, on = 'PolicyNum', how = 'left')

<h5>8.2 Count lại theo Đại lý phát hành 

<h6> 8.2.1 Tính Base

In [44]:
Sum_base_rider = base_rider
#Tính Base
Sum_base = Sum_base_rider[Sum_base_rider['Base/Rider'] == 'Base']
Sum_base = Sum_base.groupby(['Issuing Agent'])['Base/Rider'].count()
Sum_base = Sum_base.reset_index().rename(columns={'Base/Rider':'Base'})
Sum_base

,Issuing Agent,Base
0,60000482,1
1,60000717,1
2,60000727,3
3,60005660,1
4,60005897,1
...,...,...
1070,60076139,1
1071,60076140,1
1072,60076141,2
1073,60076148,2


<h6>8.2.2 Tính Rider

In [45]:
Sum_base_rider = base_rider
#Tính Base
Sum_rider = Sum_base_rider[Sum_base_rider['Base/Rider'] == 'Rider']
Sum_rider = Sum_rider.groupby(['Issuing Agent'])['Base/Rider'].count()
Sum_rider = Sum_rider.reset_index().rename(columns={'Base/Rider':'Rider'})
Sum_rider

,Issuing Agent,Rider
0,60000482,3
1,60000717,3
2,60000727,7
3,60005660,4
4,60005897,5
...,...,...
908,60076136,2
909,60076137,4
910,60076141,2
911,60076148,9


<h6>8.2.3 Tính Final Base/Rider

In [46]:
Final_Base_Rider = pd.merge(Sum_base, Sum_rider, on = 'Issuing Agent', how = 'left' )
Final_Base_Rider = Final_Base_Rider.fillna(0)
Final_Base_Rider['%Rider/Base'] = round(Final_Base_Rider['Rider'] / Final_Base_Rider['Base'], 2)
Final_Base_Rider

,Issuing Agent,Base,Rider,%Rider/Base
0,60000482,1,3.0,3.00
1,60000717,1,3.0,3.00
2,60000727,3,7.0,2.33
3,60005660,1,4.0,4.00
4,60005897,1,5.0,5.00
...,...,...,...,...
1070,60076139,1,0.0,0.00
1071,60076140,1,0.0,0.00
1072,60076141,2,2.0,1.00
1073,60076148,2,9.0,4.50


<h4> 9. Xét trạng thái Agent <h4>

In [47]:
agent_table = agent_info
agent_table['Segment'] = np.where(
agent_table['Agent_Name'].str[:5] == 'DUMMY', 'AL Trainee', 
np.where(agent_table['License_No'] == 'TAPSU', 'Agent Trainee',
np.where(agent_table['SFC'] == 'S', 'SFC',
np.where(agent_table['Area_Name'].isin(['DR1', 'HO1']), 'PHL',
np.where(agent_table['Grade'].isin(['GM', 'RM', 'DM', 'FM']), 'AL', 'FC' )
        ))))


In [48]:
day_set = Monthly_Premium[['CUTOFF', 'Start_Month', 'Date_On_Set']].drop_duplicates()
agent_T = pd.merge(agent_table, day_set, on = 'CUTOFF', how ='left')
agent_T[['Date_Appointed', 'Date_On_Set']] = agent_T[['Date_Appointed', 'Date_On_Set']].apply(pd.to_datetime) 
agent_T['Rookie'] = np.where(
(agent_T['Date_On_Set'] - agent_T['Date_Appointed']).dt.days < 30, '30',
np.where( ((agent_T['Date_On_Set'] - agent_T['Date_Appointed']).dt.days > 30) &
    ((agent_T['Date_On_Set'] - agent_T['Date_Appointed']).dt.days < 60), '60', 
np.where( ((agent_T['Date_On_Set'] - agent_T['Date_Appointed']).dt.days > 60) &
    ((agent_T['Date_On_Set'] - agent_T['Date_Appointed']).dt.days < 90), '90', '' )        
        ))

<h4> 10. Hoàn thành thông tin cá nhân của Agent <h4>

In [49]:
Individual_KPI = agent_T
Individual_KPI = Individual_KPI[['CUTOFF', 'Agent_Number', 'Agent_Name', 'Grade', 'Agent_Status', 'Date_Appointed', 
                'Segment', 'Rookie']].rename(columns={'Segment':'Segmentation'})
Individual_KPI['Period'] = Individual_KPI['CUTOFF'].str[:6]
Individual_KPI = Individual_KPI[['Period','Agent_Number', 'Agent_Name', 'Grade', 'Agent_Status', 'Date_Appointed', 
                'Segmentation', 'Rookie']]
Individual_KPI

,Period,Agent_Number,Agent_Name,Grade,Agent_Status,Date_Appointed,Segmentation,Rookie
0,202301,60060890,NGUYỄN HOÀNG ANH,FC,Enforce,2022-06-20,FC,
1,202301,60060917,PHẠM BÁ KHƯƠNG,FC,Terminated,2022-06-20,FC,
2,202301,60021398,NGUYỄN THỊ MẬU,FC,Terminated,2019-06-19,FC,
3,202301,60055999,NGUYỄN THỊ BIÊN,FC,Enforce,2022-01-25,FC,
4,202301,60075691,LÊ THỊ KHÁNH LY,FC,Enforce,2022-12-29,FC,60
...,...,...,...,...,...,...,...,...
75901,202301,60006002,NGÔ QUỐC TRUNG,FC,Terminated,2016-09-26,FC,
75902,202301,60074423,PHAN THỊ TUYẾT HƯƠNG,FC,Enforce,2022-12-12,FC,60
75903,202301,60016239,LÊ THỊ PHƯỚC,FC,Terminated,2018-08-24,FC,
75904,202301,60005479,HỒ NGỌC THÔNG,FC,Terminated,2016-08-25,FC,


<h4> 11. Ghép các Metric vào <h4>

In [109]:
PY2_KPI = pd.merge(Individual_KPI, py2_month, left_on = 'Agent_Number', right_on = 'AGENT NO', how = 'left')\
[['Period', 'Agent_Number', 'Agent_Name', 'Grade', 'Agent_Status','Date_Appointed', 'Segmentation', 'Rookie', 'AGENT NO','PY2']]
CC_KPI = pd.merge(PY2_KPI, Agent_CC, left_on ='Agent_Number', right_on ='Issuing Agent', how = 'left')\
[['Period', 'Agent_Number', 'Agent_Name', 'Grade', 'Agent_Status','Date_Appointed', 'Segmentation', 'Rookie', 'AGENT NO','PY2',
 'CC(+)', 'CC(-)']]
CC_KPI['CC'] = CC_KPI['CC(+)'] +  CC_KPI['CC(-)']
Base_Rider_KPI = pd.merge(CC_KPI, Final_Base_Rider, left_on = 'Agent_Number', right_on = 'Issuing Agent', how = 'left')
Base_Rider_KPI = Base_Rider_KPI.loc[:, Base_Rider_KPI.columns != 'Issuing Agent']
Base_Rider_KPI['Active Agent'] = np.where(Base_Rider_KPI['CC'] >0, '1', '0')
AFYP_KPI = pd.merge(Base_Rider_KPI, Complete_AFYP, left_on = 'Agent_Number', right_on = 'Servicing Agent', how = 'left')
AFYP_KPI = AFYP_KPI.loc[:, AFYP_KPI.columns != 'Servicing Agent']
AFYP_KPI['AFYP'] = AFYP_KPI['AFYP(+)'] + AFYP_KPI['AFYP(-)']
FYP_KPI = pd.merge(AFYP_KPI, Sum_FYP, left_on = 'Agent_Number', right_on = 'Servicing Agent', how = 'left')
FYP_KPI['FYP'] = FYP_KPI['FYP(+)'] + FYP_KPI['FYP(-)']
RYP = pd.merge(FYP_KPI, Sum_RYP, left_on = 'Agent_Number', right_on = 'Servicing Agent', how = 'left' )
Final_KPI = RYP
Final_KPI['Case Size'] = Final_KPI['AFYP'] / Final_KPI['CC']
Final_KPI = Final_KPI.loc[:, Final_KPI.columns !=  'Servicing Agent']
Final_KPI = Final_KPI.loc[:, Final_KPI.columns !=  'AGENT NO']
Final_KPI

,Period,Agent_Number,Agent_Name,Grade,Agent_Status,Date_Appointed,Segmentation,Rookie,PY2,CC(+),...,%Rider/Base,Active Agent,AFYP(+),AFYP(-),AFYP,FYP(-),FYP(+),FYP,RYP,Case Size
0,202301,60060890,NGUYỄN HOÀNG ANH,FC,Enforce,2022-06-20,FC,,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202301,60060917,PHẠM BÁ KHƯƠNG,FC,Terminated,2022-06-20,FC,,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202301,60021398,NGUYỄN THỊ MẬU,FC,Terminated,2019-06-19,FC,,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202301,60055999,NGUYỄN THỊ BIÊN,FC,Enforce,2022-01-25,FC,,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202301,60075691,LÊ THỊ KHÁNH LY,FC,Enforce,2022-12-29,FC,60,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75901,202301,60006002,NGÔ QUỐC TRUNG,FC,Terminated,2016-09-26,FC,,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75902,202301,60074423,PHAN THỊ TUYẾT HƯƠNG,FC,Enforce,2022-12-12,FC,60,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75903,202301,60016239,LÊ THỊ PHƯỚC,FC,Terminated,2018-08-24,FC,,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75904,202301,60005479,HỒ NGỌC THÔNG,FC,Terminated,2016-08-25,FC,,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h4> 12. Đẩy vào Database

<h5> 12.1 Tạo kết nối với SV UAT

In [92]:
connection_uri_UAT = (
    "mssql+pyodbc://PHLHCMHOUPBI01\PBIRS/PowerBI?driver=ODBC+Driver+17+for+SQL+Server"
)
conn_UAT = sa.create_engine(connection_uri_UAT)

<h5> 12.2 Tạo ra khung bảng

In [110]:
#Do kiểu float inport vào sẽ gây lỗi, chuyển hết vào kiểu str
for col in Temp_KPI_T.columns.to_list():
    Temp_KPI_T[col] = Temp_KPI_T[col].astype('str')

In [ ]:
Temp_KPI_T.to_sql("BSP_TAN_INDIVIDUAL_KPI", conn_UAT, if_exists='replace', index=False)

<h2> Note: Sau khi tạo bảng, truncate data trong SQL và chỉnh lại kiểu NVARCHAR <h2>

<h5> 12.3 Insert dữ liệu hoàn chỉnh vào Bảng

In [134]:
Temp_KPI_T = Final_KPI.fillna(value=0)
for col in Temp_KPI_T.columns.to_list():
    Temp_KPI_T[col] = Temp_KPI_T[col].astype('str')
#3.Tạo Connection với SQL
conn_toSQL = pyo.connect(driver='{ODBC Driver 17 for SQL Server}', host = 'PHLHCMHOUPBI01\PBIRS', database = 'PowerBI', trusted_connection = 'yes',
                        ansi=True,autocommit=True) #LƯU Ý: ANSI GIỮ FONT TIẾNG VIỆT
cursor = conn_toSQL.cursor()
insert_query = """\
INSERT INTO [DPO].[BSP_TAN_INDIVIDUAL_KPI] ([Period],[Agent_Number],[Agent_Name],[Grade],[Agent_Status]
      ,[Date_Appointed],[Segmentation],[Rookie],[PY2],[CC(+)],[CC(-)],[CC]
      ,[Base],[Rider],[%Rider/Base],[Active Agent],[AFYP(+)],[AFYP(-)],[AFYP],[FYP(-)],[FYP(+)],[FYP] ,[RYP],[Case Size])
      VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
"""
for index, row in Temp_KPI_T.iterrows():
    cursor.execute(insert_query,
row.Period,
row.Agent_Number,
row.Agent_Name,
row.Grade,
row.Agent_Status,
row.Date_Appointed,
row.Segmentation,
row.Rookie,
row.PY2,
row['CC(+)'],
row['CC(-)'],
row.CC,
row.Base,
row.Rider,
row['%Rider/Base'],
row['Active Agent'],
row['AFYP(+)'],
row['AFYP(-)'],
row.AFYP,
row['FYP(-)'],
row['FYP(+)'],
row.FYP,
row.RYP,
row['Case Size'])
conn_toSQL.commit()
cursor.close()